# NOTE

This is the 2nd part of the series where I explore and model the stroke prediction dataset. <br>
Here are the notebooks in the series in the correct order:

1. **[Exploratory Data Analysis (EDA)](https://www.kaggle.com/ansonnn/stroke-prediction-eda)** <br> - Exploring the dataset to derive insights about distributions and relationships between features.<br><br>
2. **Statistical Analysis** - current notebook <br> - Analyzing the normality of the features and their correlations. <br><br>
3. **[Feature Engineering and Modelling](https://www.kaggle.com/ansonnn/stroke-prediction-modelling)** <br> - Preprocessing the features and building a model for evaluation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Setup

In [ ]:
# Load necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
import numpy as np
from sklearn import preprocessing

# Enable multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last'

In [ ]:
# setting some visualization parameters
plt.style.use('seaborn')
mpl.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [ ]:
def load_preprocess_df():
    df = pd.read_csv('../input/stroke-categorical/stroke_det_cat.csv')
    cats = list(df.select_dtypes(include=['object', 'category']).columns)
    nums = list(df.select_dtypes(exclude=['object', 'category']).columns)
    
    features_to_conv = ['bmi_range', 'hypertension', 'heart_disease', 'stroke']
    cats.extend(features_to_conv)
    for feature in features_to_conv:
        if feature in nums:
            nums.remove(feature)
    print(f'Categorical variables:  {cats}')
    print(f'Numerical variables:  {nums}')
    df = df.astype({i: 'category' for i in cats})
    df = pd.concat([df[cats], df[nums]], axis=1)
    return df

In [ ]:
df = load_preprocess_df()
df.head()

In [ ]:
cat_features = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'bmi_range', 'hypertension', 'heart_disease', 'stroke']
num_features = ['age', 'avg_glucose_level', 'bmi']

In [ ]:
# Setting significance level as 0.05
ALPHA = 0.05

# Statistical Analysis on Categorical Features

**Hypothesis definition** <br>
Null hypothesis, $H_0$ : Variable A and Variable B are **independent** <br>
Alternative hypothesis, $H_1$ : Variable A and Variable B are **dependent** <br>
 <br>
Significance level, $\alpha$ is generally set to 0.05. <br>

- p-value $\le$  $\alpha$ : significant result, reject null hypothesis.
- p-value $\gt$  $\alpha$ : not significant result, fail to reject null hypothesis. <br><br>
- To analyze whether there is a significant association between two categorical features, **Pearson's chi-squared test** analysis is commonly used.

In [ ]:
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import LabelEncoder

In [ ]:
df_cat_label =  pd.concat([df.loc[:, 'gender': 'smoking_status'].apply(lambda x: LabelEncoder().fit_transform(x)),
                           df.loc[:, 'bmi_range': 'stroke']], axis=1)

df_cat_label.head()

In [ ]:
chi2_res = chi2(df_cat_label, df.stroke)
df_chi2 = pd.DataFrame({'target': 'stroke',
                        'cat_feature': df_cat_label.columns,
                        'chi2': chi2_res[0],
                        'p-value': chi2_res[1],
                        'rounded p': np.around(chi2_res[1], 3),
                        'alpha': ALPHA})
# df_chi2 = df_chi2.drop(df['cat_feature'=='stroke'], axis=0).reset_index(drop=True)
df_chi2['H0'] = df_chi2['p-value'].apply(lambda x: 'Rejected' if x <= ALPHA else 'Fail to reject')
df_chi2['relation'] = df_chi2['H0'].apply(lambda x: 'Dependent' if x=='Rejected' else 'Independent')
df_chi2.sort_values(by='chi2', ascending=False)

- When analyzing the independence between the `target` variable (`stroke` in our case) and other cat. features, <br>
Most of the features are dependent on each other except for `stroke` VS `gender`/`work_type`/`Residence_type`/`smoking_status`
- Particularly, `stroke` is very dependent on `heart_disease` and `hypertension`, with relatively very large `chi2` values

# Statistical Analysis on Numerical Features

## Visual Normality Checks

- Checking using histograms with kernel density estimate plots

In [ ]:
num_features = ['age', 'avg_glucose_level', 'bmi']

In [ ]:
df_nums = df[num_features]
df_nums.head()

In [ ]:
# Setting the overall theme for the notebook
sns.set_theme(style='darkgrid', palette='tab10', font_scale=1.1)

In [ ]:
fig = plt.figure(1, (18, 5))

for i, feature in enumerate(num_features):
    ax = plt.subplot(1, 3, i+1)
    # sns.displot(data=df_nums, x=feature, kde=True)
    sns.histplot(data=df_nums, x=feature, kde=True, bins=50)
    ax.set_xlabel(None)
    ax.set_title(f'Distribution of {feature}')
    plt.tight_layout()
plt.show()

- Already checked in EDA previously. Only `age` is similar to normal distribution.

<br>

A **log-normal distribution** is a distribution of a random variable whose logarithm is normally distributed.

Checking whether these features are log-normal or not.

In [ ]:
df_log = np.log2(df_nums)
df_log.head()

In [ ]:
fig = plt.figure(1, (18, 5))

for i, num_feature in enumerate(num_features):
    ax = plt.subplot(1, 3, i+1)
    ax.set_title(f'Distribution of log-transformed {num_feature}', size=15)
    sns.histplot(data=df_log, x=num_feature, kde=True, bins=50)
    plt.tight_layout()
plt.show()

- After log transformation, `age` has become significantly left-skewed, `avg_glucose_level` is not log-normal too.
- Only `bmi` has become visually very similar to a log-normal distribution, note that the higher peak in the middle is likely due to the median imputation for the missing values.

**Using quantile-quantile plot (Q-Q Plot) to visualize their normality**

In [ ]:
from statsmodels.graphics.gofplots import qqplot
# quantile-quantile plots on original data
fig = plt.figure(1, (18, 8))

for i, num_feature in enumerate(df_nums):
    ax = plt.subplot(2, 3, i+1)
    qqplot(df_nums[num_feature], line='s', ax=ax, markersize=5)
    ax.set_title(f'qqplot - {num_feature}')
    plt.tight_layout()
plt.show()

By looking at whether the points fall on the expected best fit line (red line, mean=0, std=1), normality can be determined. <br>
- `age` is half-way to become a normal distribution, with less data at extreme values (points at left side are above the expected best line, points at right side are below)
- `bmi` also generally seems to be normal distribution, but with some exceptions at extreme values.

## Hypothesis Tests for Normality

In [ ]:
from scipy import stats
# let's contruct a function
def shapiro_wilk_test(df, cols, alpha=0.05):
    # test the null hypothesis for columns given in `cols` of the dataframe `df` under significance level `alpha`.
    for col in cols:
        w, p = stats.shapiro(df[col])
        if p <= alpha:
            print(f"Rejected H0 under significance level {alpha}")
            print(f"{col} differs significantly from a normal distribution (W={w:.3f}, p={p:.5f})\n")
        else:
            print(f"\nFail to reject H0 due to lack of evidence under significance level {alpha}\n")
            print(f"{col} seem to be normally distributed (W={w:.3f}, p={p:.5f})")

In [ ]:
shapiro_wilk_test(df_nums, num_features)

- Shapiro-Wilk Test has a problem for detecting normality when the sample size is large (>5000), which is typical for most tests

- Trying to apply `power transform` to make the data more Gaussian-like (normal-like). Default method is `Yeo-Johnson` transform.

In [ ]:
from sklearn.preprocessing import PowerTransformer

pt = PowerTransformer()
transformed_df = pt.fit_transform(df_nums)
transformed_df = pd.DataFrame(transformed_df, columns=df_nums.columns)
transformed_df.head()

In [ ]:
fig = plt.figure(1, (18, 5))

for i, num_feature in enumerate(num_features):
    ax = plt.subplot(1, 3, i+1)
    ax.set_title(f'Distribution of power-transformed {num_feature}', size=15)
    sns.histplot(data=transformed_df, x=num_feature, kde=True, bins=50)
    plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(1, (18,8))

for i, num_feature in enumerate(df_nums):
    ax = plt.subplot(2,3,i+1)
    qqplot(transformed_df[num_feature], line='s', ax=ax, markersize=5)
    ax.set_title(f'qqplot - {num_feature}')
    plt.tight_layout()
plt.show()

- `age` is similar to before transformation, while `avg_glucose_level` has become more normal-like but still far from ideal.
- `bmi` has become almost entirely normal distribution after the power transformation.

In [ ]:
shapiro_wilk_test(transformed_df, num_features)

- Even after transforming, all of them are still deemed significantly different from normal distribution through the Shapiro-Wilk Test, <br> 
this is most likely due to the limitation of the test.
- However, visually, it is evident to tell that `age` and `bmi` can be considered normal distribution, while `avg_glucose_level` is close to normal distribution after power transformation.
- Therefore, hypothesis tests of `t-test` or `ANOVA` can still be conducted.


- Generally, Shapiro-Wilk Test is not a good way to determine whether they are Gaussian "enough" to conduct such parametric hypothesis tests. 
- Quantile-quantile plot visualization is better for this.

## Hypothesis Test for Correlation

**Hypothesis definition** <br>
Null hypothesis, $H_0$ : Variable A and Variable B are **uncorrelated** <br>
Alternative hypothesis, $H_1$ : Variable A and Variable B are **correlated** <br>
 <br>
Significance level, $\alpha$ is generally set to 0.05. <br>

- p-value $\le$  $\alpha$ : significant result, reject null hypothesis.
- p-value $\gt$  $\alpha$ : not significant result, fail to reject null hypothesis. <br><br>
- To analyze whether there is a significant association between two numerical features, **Pearson's/Spearman's correlation** are commonly used. <br>
- Such methods are for statistical hypothesis test, therefore they assume that the samples are uncorrelated (fail to reject $H_0$).

**Assumptions of pearson correlation**:
1. Both variables should have a Gaussian or Gaussian-like distribution.
2. Relationship between the variables should be linear.
3. Homoscedasticity i.e., a sequence of random variables where all its random variables have the same finite variance.

Also Pearson is quite sensitive to outliers. So in this case, Spearman's correlation is preferred since there are many outliers for `avg_glucose_level` and `bmi`.

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
from itertools import combinations

list(combinations(num_features, 2))

In [ ]:
def test_correlation(x1, x2, method='spearman', alpha=ALPHA):
    # this function returns correlation, p-value and H0 for `x1` & `x2`
    
    ALLOWED_METHODS = ['pearson', 'spearman', 'kendall']
    if method not in ALLOWED_METHODS:
        raise ValueError(f'Allowed methods are {ALLOWED_METHODS}')
        
    if method =='pearson':
        corr, p = pearsonr(x1, x2)
    elif method =='spearman':
        corr, p = spearmanr(x1, x2)
    else:
        corr, p = kendalltau(x1, x2)
    
    h0 = 'Rejected' if p <= alpha else 'Fail to reject'
    
    return corr, p, h0

In [ ]:
df_corr = pd.DataFrame(columns=['feature_1', 'feature_2', 'correlation', 'p', 'H0'])

for x, y in combinations(num_features, r=2):
    corr, p, h0 = test_correlation(df_nums[x], df_nums[y])
    
    df_corr = df_corr.append({'feature_1': x, 'feature_2': y,
                              'correlation': round(corr, 5), 'p': p, 'H0': h0}, ignore_index=True)
    
df_corr.style.highlight_max(color='navy')

- All of their null hypothesis are rejected, therefore all of the numerical features are correlated to each other. 
- With `age` and `bmi` being moderately more correlated than the other 2 combinations

# Statistical Significance Tests

**Parametric Statistical Significance Tests** <br>
1. **Student’s t-test** - It tests whether the two independent normal distributed samples have the same mean or not. <br>
2. **Analysis of Variance test (ANOVA)** - It tests whether the two or more independent normal distributed samples have the same mean or not. <br>
ANOVA serves the same purpose with t-test but for more than two variables instead of only two variables. So either apply t-test pair-wise or apply ANOVA once.  <br>
ANOVA also only tells whether all the samples have the same mean or not, it doesn't quantify which samples differ or by how much.

**Non-Parametric Statistical Significance Tests** <br>
1. **Mann-Whitney U test** - Non-parametric equivalent of **Student's t-test**. <br>
2. **Kruskal-Wallis H test** - Non-parametric equivalent of **ANOVA** (but it compares the median instead). <br>

Parametric tests are for normally distributed data, while non-parametric tests are for non-normally distributed data. <br>
However, in this case, the Shapiro-Wilk test is not a good way for determine whether they are normally distributed or not. <br>
We have already verified that the 3 numerical features `age`, `avg_glucose_level` and `bmi` are similar to normal distribution in the Q-Q plots. <br>
Therefore, it is generally good enough to apply parametric tests by inspecting the visualizations.

In [ ]:
num_features

In [ ]:
print(cat_features)

## t-tests

In [ ]:
from scipy import stats

In [ ]:
x, y = df[df.gender=='Male'].age, df[df.gender=='Female'].age

t, p = stats.ttest_ind(x, y, equal_var=False)
t, p = round(t, 5), round(p, 5)

if p <= ALPHA:
    print(f'Since p = {p} < {ALPHA}, reject H0, different Age distributions between Male and Female (t={t}, p={p})')
else:
    print(f'Since p = {p} > {ALPHA}, fail to reject H0, same Age distributions between Male and Female (t={t}, p={p}).')

- The p-value is 0.0499 which is ***very close*** to **fail to reject H0**, therefore they can actually be considered as borderline similar distribution.

In [ ]:
x, y = df[df.gender=='Male'].avg_glucose_level, df[df.gender=='Female'].avg_glucose_level

t, p = stats.ttest_ind(x, y, equal_var=False)
t, p = round(t, 5), round(p, 5)

if p <= ALPHA:
    print(f'Since p = {p} < {ALPHA}, reject H0, different avg_glucose_level distributions between Male and Female (t={t}, p={p})')
else:
    print(f'Since p = {p} > {ALPHA}, fail to reject H0, same avg_glucose_level distributions between Male and Female (t={t}, p={p}).')

In [ ]:
x, y = df[df.gender=='Male'].bmi, df[df.gender=='Female'].bmi

t, p = stats.ttest_ind(x, y, equal_var=False)
t, p = round(t, 5), round(p, 5)

if p <= ALPHA:
    print(f'Since p = {p} < {ALPHA}, reject H0, different BMI distributions between Male and Female (t={t}, p={p})')
else:
    print(f'Since p = {p} > {ALPHA}, fail to reject H0, same BMI distributions between Male and Female (t={t}, p={p}).')

In [ ]:
x, y = df[df.stroke==1].age, df[df.stroke==0].age

t, p = stats.ttest_ind(x, y, equal_var=False)
t, p = round(t, 5), round(p, 5)

if p <= ALPHA:
    print(f'Since p = {p} < {ALPHA}, reject H0, different Age distributions between Stroke and "No stroke" (t={t}, p={p})')
else:
    print(f'Since p = {p} > {ALPHA}, fail to reject H0, same Age distributions between Stroke and "No stroke" (t={t}, p={p}).')

In [ ]:
x, y = df[df.stroke==1].avg_glucose_level, df[df.stroke==0].avg_glucose_level

t, p = stats.ttest_ind(x, y, equal_var=False)
t, p = round(t, 5), round(p, 5)

if p <= ALPHA:
    print(f'Since p = {p} < {ALPHA}, reject H0, different avg_glucose_level distributions between Stroke and "No stroke" (t={t}, p={p})')
else:
    print(f'Since p = {p} > {ALPHA}, fail to reject H0, same avg_glucose_level distributions between Stroke and "No stroke" (t={t}, p={p}).')

In [ ]:
x, y = df[df.stroke==1].bmi, df[df.stroke==0].bmi

t, p = stats.ttest_ind(x, y, equal_var=False)
t, p = round(t, 5), round(p, 5)

if p <= ALPHA:
    print(f'Since p = {p} < {ALPHA}, reject H0, different BMI distributions between Stroke and "No stroke" (t={t}, p={p})')
else:
    print(f'Since p = {p} > {ALPHA}, fail to reject H0, same BMI distributions between Stroke and "No stroke" (t={t}, p={p}).')

### Summary Tables for t-test Results

In [ ]:
t_test_df = pd.DataFrame(columns=['cat_feat', 'num_feat', 'alpha', 't', 'p'])

for feature in num_features:
    t, p = stats.ttest_ind(df.loc[df.gender=='Male', feature],
                            df.loc[df.gender=='Female', feature],
                            equal_var=False)
    t, p = round(t, 3), round(p, 5)
    t_test_df = t_test_df.append({'cat_feat': 'gender', 'num_feat': feature,
                    'alpha': ALPHA, 't': t, 'p': p}, ignore_index=True)

t_test_df['H0'] = t_test_df.p.apply(lambda x: 'Rejected' if x <= ALPHA else 'Fail to reject')
t_test_df['relation'] = t_test_df.H0.apply(lambda x: 'Different' if x =='Rejected' else 'Same')

def color_fail_to_rej(value):
    if value in ('Fail to reject', 'Same'):
        return 'color:green'
    else:
        return 'cmap: Blues'
    
t_test_df.style.background_gradient(cmap = 'Blues').applymap(color_fail_to_rej)

In [ ]:
t_test_df = pd.DataFrame(columns=['cat_feat', 'num_feat', 'alpha', 't', 'p'])

for feature in num_features:
    t, p = stats.ttest_ind(df.loc[df.stroke==0, feature],
                            df.loc[df.stroke==1, feature],
                            equal_var=False)
    t, p = round(t, 3), round(p, 5)
    t_test_df = t_test_df.append({'cat_feat': 'stroke', 'num_feat': feature,
                    'alpha': ALPHA, 't': t, 'p': p}, ignore_index=True)

t_test_df['H0'] = t_test_df.p.apply(lambda x: 'Rejected' if x <= ALPHA else 'Fail to reject')
t_test_df['relation'] = t_test_df.H0.apply(lambda x: 'Different' if x =='Rejected' else 'Same')
t_test_df.style.background_gradient(cmap = 'Blues').applymap(color_fail_to_rej)

## One-way ANOVA tests

In [ ]:
smoking_age_group = df.groupby('smoking_status')['age'].unique()
smoking_age_group

In [ ]:
f_statistic, p = stats.f_oneway(*smoking_age_group)
f_statistic, p = round(t, 5), round(p, 5)

if p <= ALPHA:
    print(f'Since p = {p} < {ALPHA}, reject H0, different Age distributions among different smoking_status (F={f_statistic}, p={p})')
else:
    print(f'Since p = {p} > {ALPHA}, fail to reject H0, same Age distributions among different smoking_status (F={f_statistic}, p={p}).')

In [ ]:
bmi_range_group = df.groupby('bmi_range')['age'].unique()
f_statistic, p = stats.f_oneway(*bmi_range_group)
f_statistic, p = round(t, 5), round(p, 5)

if p <= ALPHA:
    print(f'Since p = {p} < {ALPHA}, reject H0, different Age distributions among different bmi_range (F={f_statistic}, p={p})')
else:
    print(f'Since p = {p} > {ALPHA}, fail to reject H0, same Age distributions among different bmi_range (F={f_statistic}, p={p}).')

**Conclusion**

- For the important parts, only `Male` and `Female` has similar distribution in terms of the mean values of `age` and `bmi`.
- With respect to `stroke`, all the numerical features have different distributions.

# Summary of Relationship between Categorical and Numerical Features

In [ ]:
from sklearn import feature_selection

In [ ]:
f, p = feature_selection.f_classif(df.loc[:, 'age': 'bmi'], df.gender)
f, p = np.around(f, 3), np.around(p, 5)
anova_test_cat = pd.DataFrame({
    'cat-attr': 'gender',
    'cont-attr': df.loc[:, 'age': 'bmi'].columns,
    'f': f, 
    'p': p,
    'alpha': ALPHA
})
anova_test_cat['H0'] = anova_test_cat.p.apply(lambda x: 'Rejected' if x <= ALPHA else 'Fail to reject')
anova_test_cat['relation'] = anova_test_cat.H0.apply(lambda x: 'Different' if x =='Rejected' else 'Same')
anova_test_cat.style.background_gradient(cmap = 'Blues').applymap(color_fail_to_rej)

In [ ]:
f, p = feature_selection.f_classif(df.loc[:, 'age': 'bmi'], df.stroke)
f, p = np.around(f, 3), np.around(p, 5)
anova_test_cat = pd.DataFrame({
    'cat-attr': 'stroke',
    'cont-attr': df.loc[:, 'age': 'bmi'].columns,
    'f': f, 
    'p': p,
    'alpha': ALPHA
})
anova_test_cat['H0'] = anova_test_cat.p.apply(lambda x: 'Rejected' if x <= ALPHA else 'Fail to reject')
anova_test_cat['relation'] = anova_test_cat.H0.apply(lambda x: 'Different' if x =='Rejected' else 'Same')
anova_test_cat.style.background_gradient(cmap = 'Blues').applymap(color_fail_to_rej)

**Conclusion**


- The ANOVA test results are the same with the t-test results, with little difference between the p-values.
- But it is summarized clearly in table forms here, where **only** the distributions of `gender` and `bmi` are the same.